In [1]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/dados-lanchonete-pao/DataBase po.xlsx


In [2]:
dados_pao = pd.read_excel("/kaggle/input/dados-lanchonete-pao/DataBase po.xlsx")

In [3]:
dados_pao.columns = [col.strip().lower().replace(' ', '_') for col in dados_pao.columns]

In [4]:
dados_pao['pão_comprados'] = dados_pao['pão_comprados'].fillna(dados_pao['pão_comprados'].median())
dados_pao['frequencia'] = dados_pao['frequencia'].fillna('Semanal')
dados_pao['trabalha'] = dados_pao['trabalha'].fillna('Não')
dados_pao['faltas'] = dados_pao['faltas'].fillna(0)
dados_pao['numero_de_amigos'] = dados_pao['numero_de_amigos'].fillna(dados_pao['numero_de_amigos'].mean())

In [5]:
def is_numeric(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

def formatar_interacao(x):
    if not is_numeric(x):
        if x == "Baixa":
            return 0.25
        elif x == "Média":
            return 0.5
        elif "Alta":
            return 1

    if x <= 4:
        return 0.25
    elif x >= 5 and x <= 8:
        return 0.5
    elif x >= 9 and x <= 12:
        return 1
    elif x == 0:
        return 0

dados_pao["interação_social"] = dados_pao["interação_social"].apply(formatar_interacao)

In [6]:
def classificar_cliente(linha):
    pontos = 0
    if linha['frequencia'] == 'Semanal':
        pontos += 1
    elif linha['frequencia'] == 'Quinzenal':
        pontos += 0.5
    elif linha['frequencia'] == 'Mensal':
        pontos += 0.25
    
    if linha['pão_comprados'] > 5:
        pontos += 1
    elif 3 <= linha['pão_comprados'] <= 5:
        pontos += 0.5
    elif linha['pão_comprados'] < 3:
        pontos += 0.25
    
    if linha['valor_da_ultima_compra'] > 20:
        pontos += 1
    elif 10 <= linha['valor_da_ultima_compra'] <= 20:
        pontos += 0.5
    elif linha['valor_da_ultima_compra'] < 10:
        pontos += 0.25
    
    if linha['trabalha'] == 'Sim':
        pontos += 1
    elif linha['trabalha'] == 'Não':
        pontos += 0
    
    pontos += linha['interação_social']
    
    if linha['faltas'] == 0:
        pontos += 1
    elif 1 <= linha['faltas'] <= 2:
        pontos += 0.5
    elif linha['faltas'] > 2:
        pontos -= 0.25
    
    return pontos

dados_pao['pontos'] = dados_pao.apply(classificar_cliente, axis=1)

top_clientes = dados_pao[['pessoa', 'pontos']].sort_values(by='pontos', ascending=False).head(3)

top_clientes['valor_minimo_dia'] = round(dados_pao['valor_gasto_por_semana'].min() / 7,2)  
top_clientes['valor_maximo_dia'] = round(dados_pao['valor_gasto_por_semana'].max() / 7,2)

top_clientes["probabilidade"] = (dados_pao["pontos"] / 5) * 100

resultado = top_clientes[["pessoa","probabilidade","valor_minimo_dia","valor_maximo_dia"]]
resultado.columns = ["Nome pessoa","Probabilidade de compra (%)","Valor mínimo de compra","Valor máximo de compra"]

resultado.to_csv("probabilidade_pao.csv")
resultado

,Nome pessoa,Probabilidade de compra (%),Valor mínimo de compra,Valor máximo de compra
35,Renata Ribeiro,95.0,1.43,4.86
30,Rafael Rocha,95.0,1.43,4.86
12,Thiago Ramos,95.0,1.43,4.86
